In [7]:
import PyPDF2
from PyPDF2 import PdfReader
from langchain.llms import OpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain

from IPython.display import display, HTML

from util import local_settings

display(HTML(f"""
✅ Libraries loaded successfully <br>
✅ OpenAI Key loaded (...{local_settings.OPENAI_API_KEY[10:-15]}...)
"""))

sk-1vZwyXUPgRnFwjGVxTgzT3BlbkFJ3FPqf


In [8]:
pdf_data = PdfReader("C:\\Users\\catio\\Downloads\\meds.pdf")
pdf_text = ""

for i, page in enumerate(pdf_data.pages):
    text = page.extract_text()
    if text:
        pdf_text += text

#print(len(pdf_text))

In [10]:
llm = OpenAI(model_name ="gpt-3.5-turbo-instruct", openai_api_key = local_settings.OPENAI_API_KEY)

medicine = "varfarina"

prompt = f" When should I not take {medicine}? Answer 'I don't know' if you don't know."

response = llm(prompt, temperature = 0)

print("----- response -----")
print(response.replace("Langchain", ""))

----- response -----


I don't know.


In [4]:
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 700,
    chunk_overlap = 100
)

final_data = text_splitter.split_text(pdf_text)

print(f"""
    # of Chunks: {len(final_data)}
    Chunk 0: {final_data[0]}
    Chunk 1: {final_data[1]}
""")

Created a chunk of size 702, which is longer than the specified 700
Created a chunk of size 778, which is longer than the specified 700
Created a chunk of size 794, which is longer than the specified 700
Created a chunk of size 774, which is longer than the specified 700
Created a chunk of size 707, which is longer than the specified 700
Created a chunk of size 1145, which is longer than the specified 700
Created a chunk of size 1459, which is longer than the specified 700
Created a chunk of size 1601, which is longer than the specified 700
Created a chunk of size 1451, which is longer than the specified 700
Created a chunk of size 1268, which is longer than the specified 700
Created a chunk of size 1631, which is longer than the specified 700
Created a chunk of size 1586, which is longer than the specified 700
Created a chunk of size 1454, which is longer than the specified 700
Created a chunk of size 1261, which is longer than the specified 700
Created a chunk of size 1742, which is 


    # of Chunks: 1437
    Chunk 0: APROVADO EM 
 06-04-2023  INFARMED 
 
 Folheto informativo: Informação para o doente 
 
Varfine 5 mg comprimidos 
varfarina 
  
Leia com atenção todo este folheto antes de começar a tomar este medicamento, pois
    Chunk 1: varfarina 
  
Leia com atenção todo este folheto antes de começar a tomar este medicamento, pois 
contém informação importante para si. - Conserve este folheto. Pode ter necessidade de o ler novamente.  - Caso ainda tenha dúvidas, fale com o seu médico ou farmacêutico. - Este medicamento foi receitado apenas para si. Não deve dá-lo a outros. O medicamento pode ser-lhes prejudicial mesmo que apresentem os mesmos sinais de doença. - Se tiver quaisquer efeitos indesejáveis, incluindo possíveis efeitos indesejáveis não indicados neste folheto, fale com o seu médico ou farmacêutico ou enfermeiro. Ver secção 4.  O que contém este folheto: 
1. O que é Varfine e para que é utilizado



In [5]:
embeddings = OpenAIEmbeddings()
# emmbedinng the final_data, and group them according to similarity (Faiss- VectorDB)
document_searcher = FAISS.from_texts(final_data, embeddings)
# retrieves all the chunks (chain_type = stuff) that are similar to the question
chain = load_qa_chain(OpenAI(), chain_type="stuff")


In [6]:
medicine = "varfarina"

prompt = f" When should I not take {medicine}? Answer 'I don't know' if you don't know."

docs =  document_searcher.similarity_search(prompt)
result = chain.run(input_documents=docs, question=prompt)

print("--- 🤖 RESULT ---")
print(result)

--- 🤖 RESULT ---
 You should not take Varfine if you have an allergy to the active substance or any of the other components of this medication, if you are pregnant (especially during the first and last three months of pregnancy) or think you might be pregnant or have had a baby in the last 48 hours, if you have or have had any bleeding problems, if you have recently had a stroke (bleeding in the brain), or if you have undergone surgery in the last 72 hours or will be undergoing surgery in the next 72 hours.
